<img src="https://brand.umich.edu/assets/brand/style-guide/logo-guidelines/U-M_Logo-Horizontal-Hex.png" alt="Drawing" style="width: 300px;" align="left"/><br>
    
## Week 4: Dashboard

For this notebook, you are building a prototype dashboard. The specific user persona we have in mind for you to design around is a non-technical instructor who is relatively new to teaching online and has moderate to low data literacy skills. 

The instructor, as we have mentioned in previous Notebooks, is looking to run a report about 25% of the way through the course in order to identify who he or she should do a formal check-in with.
 
To help you in building your dashboard, this notebook provides a brief introduction to the [jupyter-dash extension](https://github.com/plotly/jupyter-dash) for building a [Plotly Dash](https://github.com/plotly/dash) app within Jupyter environments. You can check out the documentation and some tutorials for Plotly Dash [here](https://dash.plotly.com/).

Resources:
* [A medium blog - Introducing JupyterDash](https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e)
* [Plotly Dash documentation](https://dash.plotly.com/)
* [A Youtube tutorial on dash](https://www.youtube.com/watch?v=hSPmj7mK6ng)
* [Dash gallery](https://dash-gallery.plotly.host/Portal/)

You can also check out [OU Analyse](https://analyse.kmi.open.ac.uk/), screenshot below, as an example of how the authors of the dataset have developed a dashboard for their institution. You can request a demo of the dashboard by entering your email.
![OU Analyse](https://analyse.kmi.open.ac.uk/resources/images/project_info/screenshot_01.png)

In [2]:
# !pip install jupyter-dash  ## should already be installed

import pandas as pd
import numpy as np

# Import jupyter dash
from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import dcc
from dash import html

# Import plotly
import plotly.graph_objs as go
import plotly.express as px

# Set up jupyter proxy
JupyterDash.infer_jupyter_proxy_config()

In [3]:
lr = pd.read_csv('assets/learning_resources.csv')
qt = pd.read_csv('assets/quizzes_tests.csv')
si = pd.read_csv('assets/student_info_pred.csv')
df = pd.read_csv('assets/country_indicators.csv')

# A quick tutorial

In essence, a plotly-dash dashboard consists of 3 components:
* The **dash components** (e.g., dropdown, slider, checklist, etc.). See the documentation [here](https://dash.plotly.com/dash-core-components)
* The **plotly** graphs (e.g., linegraph, scatter plot, heatmap, etc.). See the documentation [here](https://plotly.com/python/)
* The **callback** to connects the dash components to plotly graphs, making it an interactive dashboard. See the documentation [here](https://dash.plotly.com/basic-callbacks) 



## Step 1. The Dash components (i.e. layouts)

In [4]:
score = qt.groupby('assignment_name')['score'].agg({np.mean,np.median, np.std}).reset_index(drop = False)

In [5]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = score['assignment_name'].unique()

# Step 1

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is the title"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='Quiz 1' # the default is code_module AAA
            )])

# Run the app
app.run_server(mode="inline", port = 8100)

# You will see we have the dropdown menu but nothing happens yet

## Step 2 & 3. Callback and create plotly graph

In [6]:
# Let's import a sample dataframe
#df = pd.read_csv('assets/course_passrate.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = score['assignment_name'].unique()

# Step 1

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a bar chart"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='Quiz 1' # the default
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(assignment_name):
    filtered_score = score[(score.assignment_name == assignment_name)]
    figure = px.bar(filtered_score, x = 'assignment_name', y = 'mean')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode = "inline", port = 8101)

In [7]:
# Let's import a sample dataframe
#df = pd.read_csv('assets/course_passrate.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of assignment_name
available_indicators1 = score['assignment_name'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a scatter plot"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a checklist based on assignment_name
            dcc.Checklist(
                    id = 'checklist',
                    options=[{'label': i, 'value': i} for i in available_indicators1],
                    value=available_indicators1 # Default values contain all assignment_name
            )
])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('checklist', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(assignment_name):
    figure = px.scatter(score, 
                     x="mean", 
                     y="std", 
                     color="assignment_name",
                     hover_name='assignment_)n')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode="inline", port = 8102)

# Build a demo app

In [8]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

available_indicators = df['Indicator Name'].unique()

# Design app layout
app.layout = html.Div([
    html.Div([

        html.Div([
            # Create a dropdown menu on x-axis
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            
            # Create a multiple choice menu on x-axis
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            # Create a dropdown menu on y-axis
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            # Create a multiple choice menu on y-axis
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),
    
    # Create a dashboard which consists of a scatter plot, x-time-series, and y-time-series
    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),
    
    # Create a slider on year
    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

# Call back function
# It will take the 5 inputs and produce 1 output which is the scatter plot figure
@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])

# Define update_graph function
# It basically filters the df based on users' input (e.g. dropdown menu, slider, multiple choice)
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [dict(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }


def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

# Call back for time-series graph on x-axis
@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)

# Call back for time-series graph on y-axis
@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

In [9]:
# Run the app inside jupyter notebook
app.run_server(mode="inline", port =8051)

# Building Your Dashboard (40 points)

Your final product will need to have the following capabilities:

1. (10 points) View multiple students' predicted probabilities of failing the course
2. (10 points) View a student's quiz/test performances alongside a meaningful reference like a course average for a given quiz/test
3. (10 points) View a student's learning resource use alongside a meaningful reference like a course average for a given resource
4. (10 points) Intergrate the above three capabilties into a single, functional dashboard

This notebook is broken out into four sections, three to build individual components and one to integrate them. The final Integrated Dashboard cell will be graded and must include all three specified capabilities. If it does not, points will be awarded based on progress made in each Component cell.

There is a new column in `student_info.csv`, `fail_pred,` that represents the predicted probability of failure for each student from a baseline model.

**Note:** All quizzes, tests, and resources should only include information from on or before day 60 in the course.

### Dictionary
- **student_info.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **gender** = character; M = "male", F = "female"
    - **highest_education** = character; “Some Graduate”, “Some Higher Education”, “High School + Advanced Placement”, “High School”, “No Formal Quals” (Categories ordered from highest documented education level attained to lowest documented education level attained)
    - **disability** = character; Y = "yes", N = "no"
    - **final_result** = character; "Fail", "Pass"
    - **fail_pred** = numeric; predicted probability from sample model


- **quizzes_tests.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **assignment_name** = character; name of graded assignment (Quiz 1-7, Test 1-6, Final Exam)
    - **due_date** = numeric; date assignment was due (indexed as count in days from start of course, i.e., day 0)
    - **weight** = numeric; weight multiplied by score when generating final grade (weight * score / 100)
    - **date_submitted** = numeric; date student submitted assigned (indexed as count in days from start of course, i.e., day 0, NaN means students did not submit assignment)
    - **score** = numeric; score student earned on assignment (0 means students did not submit assignment)


- **learning_resources.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **activity_type** = character; overarching label for learning activity students can access (“course_homepage”, “course_page”, “forum”, ‘resource”, “wiki”)
    - **activity_id** = numeric; unique identifier for specific learning activity student accessed within overacting `activity_type`
    - **date** = numeric; date student accessed specific `acitivity_id`  (indexed as count in days from start of course, i.e., day 0)
    - **sum_click** = numeric; count of clicks for `activity_id` on date

## Component 1
1. (10 points) View multiple students' predicted probabilities of failing the course

In [10]:
#Copied the dataframe, so I would not change the original. Changed the integer value of id_students to a string value.
si1=si.copy()
si1['id_student']= si1['id_student'].astype(str)
si1[si1['id_student']=='41060']

,pred_fail,id_student,gender,highest_education,disability,final_result
0,0.133394,41060,M,Some Higher Education,N,Fail


In [11]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = si1['id_student'].unique()

# Step 1

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("Predicted Probability of Failing the Course"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='41060' # the default is code_module AAA
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
# This graph is a bar graph that allows the user to choose a student (Based on their ID) and see what the predicted
# Probability of Failing the course is.
def update_figure(id_student):
    filtered_score = si1[si1['id_student']==id_student]
    figure = px.bar(filtered_score, x = 'id_student', y = 'pred_fail',text_auto=True)
    return figure 
    
# Run the app
app.run_server(mode = "inline", port = 8101)

## Component 2

2. (10 points) View a student's quiz/test performances alongside a meaningful reference like a course average for a given quiz/test

**Note:** All quizzes, tests, and resources should be from on or before day 60 in the course.

In [12]:
#Took a look at the first 5 rows of the qt DataFrame.
qt.head()

,id_student,assignment_name,due_date,weight,date_submitted,score
0,41060,Quiz 1,23,2.0,25.0,77
1,41060,Test 1,25,7.5,24.0,85
2,41060,Quiz 2,51,3.0,54.0,94
3,41060,Test 2,53,10.0,53.0,86
4,41060,Quiz 3,79,3.0,81.0,94


In [13]:
#Restricted the data by using assignments due in the first 60 days of the course.
qt1=qt[qt['due_date']<=60]
#Changed the na values to 0
qt1.fillna(0)
#Created a dictionary to help organize the means of each assessment.
dictScore={}
#looped through each asssessment to calculate their means.
for assignment in qt1['assignment_name'].unique():
    qtAssign=qt1[qt1['assignment_name']==assignment]
    dictScore[assignment]=[round(np.mean(qtAssign['score']),2)]
#Just created trial to make sure my code works.
trial=qt1[qt1['id_student']==41060]
trial['Type']='Score'
trial=trial[['assignment_name','score','Type']]
#Combined the the DataFrame with the mean scores to visualize this data easily.
dfMean = (pd.DataFrame.from_dict(dictScore)).T.reset_index()
dfMean['Type']='Mean'
dfMean.columns=['assignment_name','score','Type']
pd.concat([dfMean,trial]).reset_index(drop=True)

,assignment_name,score,Type
0,Quiz 1,66.57,Mean
1,Test 1,71.08,Mean
2,Quiz 2,75.22,Mean
3,Test 2,66.99,Mean
4,Quiz 1,77.00,Score
5,Test 1,85.00,Score
6,Quiz 2,94.00,Score
7,Test 2,86.00,Score


In [14]:
# Let's import a sample dataframe
#df = pd.read_csv('assets/course_passrate.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = qt1['id_student'].unique()

# Step 1

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("Score Compared to Mean Score on Given Assignment"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value=41060 # the default is code_module AAA
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
# This graph produces a line graph that has both the mean scores and the students score the user chose.
def update_figure(id_student):
    student=qt1[qt1['id_student']==id_student]
    student['Type']='Score'
    student=student[['assignment_name','score','Type']]
    score=pd.concat([dfMean,student]).reset_index(drop=True)
    figure = px.line(score, x = 'assignment_name', y = 'score',color='Type',text="score")
    figure.update_traces(textposition="top right")
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode = "inline", port = 8101)

## Component 3

3. (10 points) View a student's learning resource use alongside a meaningful reference like a course average for a given resource

**Note:** All quizzes, tests, and resources should be from on or before day 60 in the course.

In [15]:
#Restricted the resource data for just the first 60 days of the course.
lr1=lr[lr['date']<=60]

In [16]:
#grouped the data by summing the sum_click based on activity type and id_student.
lrSum = lr1.groupby(['activity_type','id_student'],sort=False)['sum_click'].sum().reset_index()
lrSum

,activity_type,id_student,sum_click
0,resource,420388,89
1,course_homepage,420388,379
2,course_page,420388,161
3,course_homepage,415695,244
4,course_page,409109,124
...,...,...,...
3919,wiki,41060,11
3920,wiki,2626179,13
3921,wiki,474133,3
3922,wiki,2281292,3


In [17]:
#Created a dictionary to store all the mean data for each of the resources.
dictlr={}
for activity in lrSum['activity_type'].unique():
    lrActivity=lrSum[lrSum['activity_type']==activity]
    dictlr[activity]=[round(np.mean(lrActivity['sum_click']),2)]
dflrMean = (pd.DataFrame.from_dict(dictlr)).T.reset_index()
dflrMean['Type']='Mean'
dflrMean.columns=['activity_type','sum_click','Type']
dflrMean

,activity_type,sum_click,Type
0,resource,78.69,Mean
1,course_homepage,178.65,Mean
2,course_page,193.18,Mean
3,forum,160.04,Mean
4,wiki,38.58,Mean


In [18]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = lrSum['id_student'].unique()

# Step 1

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("Number of Resource Clicks Compared to the Mean for the Course"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value=41060 # the default is code_module AAA
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
#Created a line graph that compares the student chosen resource activity compared to the mean of the class.
def update_figure(id_student):
    student=lrSum[lrSum['id_student']==id_student]
    student['Type']='Score'
    student=student[['activity_type','sum_click','Type']]
    score=pd.concat([dflrMean,student]).reset_index(drop=True)
    score = score.sort_values(by="activity_type")
    figure = px.line(score, x = 'activity_type', y = 'sum_click',color='Type',text="sum_click")
    figure.update_traces(textposition="top right")
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode = "inline", port = 8101)

---

## Integrated Dashboard

4. (10 points) Intergrate the above three capabilties into a single, functional dashboard

In [19]:
#Turned the id_student data from interger to string.
qt1['id_student']=qt1['id_student'].astype(str)
lrSum['id_student']=lrSum['id_student'].astype(str)

In [20]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = si1['id_student'].unique()

# Design app layout
# Created a button to choose the metric that the user wants.
#Created a dropdown menu to choose the student ID the user wants to access.
app.layout = html.Div([
            html.H1("Score Compared to Mean Score on Given Assignment"),
            # Create a multiple choice menu on x-axis
            dcc.RadioItems(
                id='graphType',
                options=[{'label': i, 'value': i} for i in ['Prob_Failure', 'Scores','Sum_Clicks']],
                labelStyle={'display': 'inline-block'},
                value='Prob_Failure',
            ),            
            
            dcc.Dropdown(
                id='student_id',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='41060',
            ),
            dcc.Graph(id='graph-with-dropdown')])

@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('graphType', 'value'),dash.dependencies.Input('student_id', 'value')])

#Created conditionals based on the button the user chose.
#Created graphs for each metric using the functions from the previous cells.
def update_graph(scoreType,students):
    if scoreType=='Prob_Failure':
        filtered_score = si1[si1['id_student']==students]
        figure = px.bar(filtered_score, x = 'id_student', y = 'pred_fail',text_auto=True)
        return figure
    elif scoreType=='Scores':
        student=qt1[qt1['id_student']==students]
        student['Type']='Score'
        student=student[['assignment_name','score','Type']]
        score=pd.concat([dfMean,student]).reset_index(drop=True)
        figure = px.line(score, x = 'assignment_name', y = 'score',color='Type',text="score")
        figure.update_traces(textposition="top right")
        return figure
    elif scoreType=='Sum_Clicks':
        student=lrSum[lrSum['id_student']==students]
        student['Type']='Score'
        student=student[['activity_type','sum_click','Type']]
        score=pd.concat([dflrMean,student]).reset_index(drop=True)
        score = score.sort_values(by="activity_type")
        figure = px.line(score, x = 'activity_type', y = 'sum_click',color='Type',text="sum_click")
        figure.update_traces(textposition="top right")
        return figure       

app.run_server(mode = "inline", port = 8101)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File /opt/conda/lib/python3.8/site-packages/plotly/express/_chart_types.py:66, in scatter(
    data_frame=   assignment_name        std  median       mean...
13          Test 6  34.210328    35.0  36.351784,
    x='mean',
    y='std',
    color='assignment_name',
    symbol=None,
    size=None,
    hover_name='assignment_)n',
    hover_data=None,
    custom_data=None,
    text=None,
    facet_row=None,
    facet_col=None,
    facet_col_wrap=0,
    facet_row_spacing=None,
    facet_col_spacing=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    orientation=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    color_continuous_scale=None,
    range_color=None,
    color_continuous_midp

## 2. Interpret (10 points)

After developing your integrated dashboard, provide your thoughs on the following questions:

2.1 How well will your integrated dashboard help an instructor identify which students to check-in with, and why? (5 points)

This dashboard will allow the instructor to see whether a student's scores are on par with his/her fellow classmates, and it will help an instructor determine whether accessing the resources is an issue for each student. For example, an instructor can first look at the students who have a high probability of failing this course. After identifying these students, the instructor can take a look at their assessment scores and see how they are progressing compared to the rest of the class. Lastly, the instructor can look at the resource metric to see if one of the possible reasons for the low assessment scores is the lack of utilizing the course tools. This will allow the teacher to take actionable steps to support the student by motivating them to access the resources, or even specifically stating which resources could help improve their assessment scores.

2.2 Based on the readings and videos, what would you recommend as improvements to your dashboard, and why? (5 points)

One of the major drawbacks to the current dashboard is the inability to sort the data (for example, ascending or descending order of predicted failure). It would be beneficial to be able to see which students are highest in failure prediction. As of now, the instructor would have to sift through each student to identify the ones at risk. Also, it would be useful to have several more metrics added to the dashboard to determine what might be causing this student to be unsucessful in the course. These can include some features from the previous weeks' assignments, which include completion rate and whether they are submitting work on time.  With these added features, the instructor will have a better understanding of the students' struggles and challenges and be able to support them on a more individual basis. You could take the dashboard even one step further and actually recommend specific actions to the instructor based on these metrics. We could create a summary table that the instructors can access, which will give them the best next steps. This could be created on a course-by-course basis with the instructor being part of the building process.
